# User Recommend

- User recommend toma como parámetro un año particular, y retorna el top 3 de juegos MÁS recomendados por usuarios para dicho año.

- Para evitar consultas lentas, crearemos una tabla auxiliar que sera agrupada por año y jugador, donde agruparemos los generos en un str y sumaremos el tiempo jugado del usuario.

In [27]:
import pandas as pd
import os

path_reviews = os.path.join('..','data','clear','user_reviews.csv.gz')
path_steam_games = os.path.join('..','data','clear','steam_games.csv.gz')


## Carga de data

In [32]:
## Cargamos la data necesaria para solucionar este problema.
# steam_colums = ['id','app_name']
steam_colums = ['id','app_name','release_year']

steam_games = pd.read_csv(path_steam_games, usecols = steam_colums)

## renombraremos la columna id por steam_id
steam_games.rename(columns={'id':'steam_id'}, inplace=True)


# reviews_cols = ['item_id', 'sentiment_analysis', 'recommend', 'year_posted']
reviews_cols = ['item_id', 'sentiment_analysis', 'recommend']
user_reviews = pd.read_csv(path_reviews, compression='gzip',usecols=reviews_cols,lineterminator='\n')



In [33]:

steam_games.head(4)

,app_name,steam_id,release_year
0,Lost summoner kitty,761140.0,2018
1,Ironbound,643980.0,2018
2,Real pool 3d - poolians,670290.0,2017
3,弹炸人2222,767400.0,2017


## Creación de tabla consulta

- En esta sección combinaremos user_reviews con user_items, por medio de sus ids.

- Vamos a hacer un join con steam_games y user_reviews,
dado que necesitamos los juego mas recomendados por usuarios, necesitamos los usuarios que hicieron almenos un comentario. es por esto que utilizaremos un inner join para no tener comentarios de usuarios desconocidos y usuarios que no realizaron comentarios



In [34]:
user_reviews.head(4)

,item_id,recommend,sentiment_analysis
0,1250,1,2
1,22200,1,1
2,43110,1,2
3,251610,1,2


In [36]:
## Con fecha de steam
user_reviews[(user_reviews['item_id'] == 33930) & (user_reviews['recommend']==1)& ((steam_games['release_year'] == 2010) )]

,item_id,recommend,sentiment_analysis


In [24]:
user_reviews[(user_reviews['item_id'] == 33930) & (user_reviews['recommend']==1)& ((user_reviews['year_posted'] == 2010) )]

,item_id,year_posted,recommend,sentiment_analysis


In [37]:
mask = (user_reviews['recommend'] == 1 ) & (user_reviews['sentiment_analysis'].isin([2,1])) 
user_reviews_recommend = user_reviews[mask]

In [38]:
steam_reviews = steam_games.merge(user_reviews_recommend,
                              left_on='steam_id', 
                              right_on= 'item_id',
                              how='inner').drop(columns=['steam_id','item_id'])

In [40]:
steam_reviews[(steam_reviews['app_name'] == 'Arma 2: operation arrowhead') & (steam_reviews['release_year']==2010)]

,app_name,release_year,recommend,sentiment_analysis
30557,Arma 2: operation arrowhead,2010,1,2
30558,Arma 2: operation arrowhead,2010,1,1
30559,Arma 2: operation arrowhead,2010,1,2
30560,Arma 2: operation arrowhead,2010,1,2
30561,Arma 2: operation arrowhead,2010,1,2
...,...,...,...,...
30625,Arma 2: operation arrowhead,2010,1,2
30626,Arma 2: operation arrowhead,2010,1,2
30627,Arma 2: operation arrowhead,2010,1,1
30628,Arma 2: operation arrowhead,2010,1,2


In [8]:
tabla_endpoint_3 = steam_reviews[['year_posted','app_name','recommend']].groupby(['year_posted','app_name']).sum()

In [11]:
tabla_endpoint_3

recommend
year_posted app_name                            
2010        Alien swarm                        4
            Amnesia: the dark descent          1
            Arcania                            1
            Audiosurf                          1
            Chime                              1
...                                          ...
2015        Zombi                              1
            Zombie army trilogy                5
            Zombie exodus                      1
            Zoombinis                          3
            Астролорды: облако оорта           1

[3653 rows x 1 columns]

## Carga de Data

In [13]:
path_endpoint_3 = os.path.join('..','data','clear','03_users_recommend.csv.gz')

tabla_endpoint_3.to_csv(path_endpoint_3,compression='gzip')

##Prueba

In [26]:
df_filtrado = steam_reviews[(steam_reviews['year_posted'] == 2010) &(steam_reviews['recommend'] == 1) & (steam_reviews['sentiment_analysis'].isin([1, 2]))]
df_filtrado[df_filtrado['app_name']=="Sid meier's civilization® v"]

,app_name,year_posted,recommend,sentiment_analysis
2853,Sid meier's civilization® v,2010,1,2


## Lectura de tabla consulta

In [14]:
table_3 = pd.read_csv(path_endpoint_3)

In [15]:
table_3

,year_posted,app_name,recommend
0,2010,Alien swarm,4
1,2010,Amnesia: the dark descent,1
2,2010,Arcania,1
3,2010,Audiosurf,1
4,2010,Chime,1
...,...,...,...
3648,2015,Zombi,1
3649,2015,Zombie army trilogy,5
3650,2015,Zombie exodus,1
3651,2015,Zoombinis,3


In [169]:
year = 2015


In [164]:
table_3[['app_name','recommend']][table_3['year_posted']==year].groupby('app_name').sum()

,recommend
app_name,
Alien swarm,4
Amnesia: the dark descent,1
Arcania,1
Audiosurf,1
Chime,1
Civilization iv: beyond the sword,1
Counter-strike: source,2
Day of defeat: source,1
Doom ii,1


In [162]:
top_3 = table_3[['app_name','recommend']][table_3['year_posted']==year].groupby('app_name').sum().nlargest(3,'recommend').reset_index()




In [163]:
top_3

,app_name,recommend
0,Team fortress 2,10
1,Killing floor,6
2,Alien swarm,4


In [157]:
# Crea el diccionario con los puestos y los nombres de las aplicaciones
diccionario_resultado = {f'Puesto{i+1}': juego for i, juego in enumerate(top_3['app_name'])}

In [158]:
[diccionario_resultado]

[{'Puesto1': 'Team fortress 2',
  'Puesto2': 'Killing floor',
  'Puesto3': 'Alien swarm'}]

## Funcion UsersRecommend

In [131]:
def user_reviews_recommend(year: int):
  path_endpoint_3 = os.path.join('..','data','clear','03_users_recommend.csv.gz')
  table_3 = pd.read_csv(path_endpoint_3)
  
  top_3 = table_3[['app_name','recommend']][table_3['year_posted']==year].groupby('app_name').sum().nlargest(3,'recommend').reset_index()
  
  diccionario_resultado = {f'Puesto{i+1}': juego for i, juego in enumerate(top_3['app_name'])}
  
  return [diccionario_resultado]
  

In [132]:
user_reviews_recommend(2010)

[{'Puesto1': 'Team fortress 2',
  'Puesto2': 'Killing floor',
  'Puesto3': 'Alien swarm'}]